In [1]:
import pandas as pd


# 1. ĐỌC DỮ LIỆU

patients = pd.read_csv(r"D:\case13\patient_info.csv")
vaccines = pd.read_csv(r"D:\case13\vaccination_record.csv")
reactions = pd.read_csv(r"D:\case13\reaction_log.csv")


# 2. CHUẨN HÓA patient_id

for df in [patients, vaccines, reactions]:
    df["patient_id"] = (
        df["patient_id"]
        .astype(str)
        .str.strip()
        .str.upper()
    )



In [2]:
import pandas as pd


# 1. ĐỌC DỮ LIỆU

patients = pd.read_csv(r"D:\case13\patient_info.csv")
vaccines = pd.read_csv(r"D:\case13\vaccination_record.csv")
reactions = pd.read_csv(r"D:\case13\reaction_log.csv")


# 2. CHUẨN HÓA patient_id

for df in [patients, vaccines, reactions]:
    df["patient_id"] = (
        df["patient_id"]
        .astype(str)
        .str.strip()
        .str.upper()
    )


# 3. FULL OUTER MERGE

# Merge patient_info + vaccination_record
pv_outer = patients.merge(
    vaccines,
    on="patient_id",
    how="outer",
    indicator="merge_patient_vaccine"
)

# Merge thêm reaction_log
full_outer_merge = pv_outer.merge(
    reactions,
    on="patient_id",
    how="outer",
    indicator="merge_reaction"
)

# =========================
# 4. PHÁT HIỆN LỆCH DỮ LIỆU
# =========================

# 4.1 Chỉ có patient_info (chưa tiêm)
only_patient = full_outer_merge[
    full_outer_merge["merge_patient_vaccine"] == "left_only"
]

# 4.2 Có tiêm nhưng không có phản ứng
no_reaction = full_outer_merge[
    (full_outer_merge["merge_patient_vaccine"] == "both") &
    (full_outer_merge["merge_reaction"] == "left_only")
]

# 4.3 Có vaccine hoặc reaction nhưng thiếu patient_info
missing_patient_info = full_outer_merge[
    full_outer_merge["merge_patient_vaccine"] == "right_only"
]

# =========================
# 5. THỐNG KÊ TỔNG QUAN
# =========================
summary = (
    full_outer_merge
    .groupby(["merge_patient_vaccine", "merge_reaction"])
    .size()
    .reset_index(name="count")
)


# 6. IN KẾT QUẢ

print("=== BẢNG FULL OUTER MERGE ===")
print(full_outer_merge.head(), "\n")

print("=== BỆNH NHÂN CHỈ CÓ HỒ SƠ (CHƯA TIÊM) ===")
print(only_patient.head(), "\n")

print("=== BỆNH NHÂN CÓ TIÊM NHƯNG KHÔNG CÓ PHẢN ỨNG ===")
print(no_reaction.head(), "\n")

print("=== BỆNH NHÂN THIẾU THÔNG TIN HỒ SƠ ===")
print(missing_patient_info.head(), "\n")

print("=== THỐNG KÊ LỆCH DỮ LIỆU ===")
print(summary)

=== BẢNG FULL OUTER MERGE ===
  patient_id age_group   gender vaccine_type dose_number vaccination_date  \
0      BN500      6-17  Female      covid19            2       2024-04-15   
1      BN500      6-17  Female           MMR         1st       2024-04-04   
2      BN500      6-17  Female      covid19            2       28/02/2024   
3      BN500      6-17  Female           DPT           1       19/01/2024   
4      BN500      6-17  Female         d.p.t       three       2024-05-11   

  merge_patient_vaccine reaction_type severity report_date merge_reaction  
0                  both      đau tay         3  27/04/2024           both  
1                  both      đau tay         3  27/04/2024           both  
2                  both      đau tay         3  27/04/2024           both  
3                  both      đau tay         3  27/04/2024           both  
4                  both      đau tay         3  27/04/2024           both   

=== BỆNH NHÂN CHỈ CÓ HỒ SƠ (CHƯA TIÊM) ===
   pat

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_18736\3255269105.py:65: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby(["merge_patient_vaccine", "merge_reaction"])


In [3]:
# =========================
# MERGE: patient_info + vaccination_record
# =========================
patients_vaccines = patients.merge(
    vaccines,
    on="patient_id",
    how="left",
    indicator=True
)
print("\nKết quả merge patient + vaccine (_merge counts):")
print(patients_vaccines["_merge"].value_counts())
display(patients_vaccines.head())


Kết quả merge patient + vaccine (_merge counts):
_merge
both          120
left_only       1
right_only      0
Name: count, dtype: int64


,patient_id,age_group,gender,vaccine_type,dose_number,vaccination_date,_merge
0,BN500,6-17,Female,covid19,2,2024-04-15,both
1,BN500,6-17,Female,MMR,1st,2024-04-04,both
2,BN500,6-17,Female,covid19,2,28/02/2024,both
3,BN500,6-17,Female,DPT,1,19/01/2024,both
4,BN500,6-17,Female,d.p.t,three,2024-05-11,both


In [4]:
import pandas as pd
import numpy as np
from IPython.display import display

# =========================
# 1. ĐỌC DỮ LIỆU & CHUẨN HOÁ
# =========================
patients = pd.read_csv(r"D:\case13\patient_info.csv")
vaccines = pd.read_csv(r"D:\case13\vaccination_record.csv")
reactions = pd.read_csv(r"D:\case13\reaction_log.csv")

# Chuẩn hoá patient_id
for df in [patients, vaccines, reactions]:
    df["patient_id"] = df["patient_id"].astype(str).str.strip().str.upper()

print("Loaded datasets:")
print("patients:", patients.shape)
print("vaccines:", vaccines.shape)
print("reactions:", reactions.shape)

Loaded datasets:
patients: (40, 3)
vaccines: (120, 4)
reactions: (80, 4)


In [5]:

# MERGE THÊM: reaction_log -> final_df

final_df = patients_vaccines.merge(
    reactions,
    on="patient_id",
    how="left",
    indicator="reaction_merge"
)
print("\nKết quả merge thêm reaction :")
print(final_df["reaction_merge"].value_counts())
display(final_df.head())

# PHÁT HIỆN THIẾU DỮ LIỆU

missing_vaccination = final_df[final_df["_merge"] == "left_only"][["patient_id"]].drop_duplicates()
missing_reaction = final_df[final_df["reaction_merge"] == "left_only"][["patient_id"]].drop_duplicates()

print("\nBệnh nhân thiếu dữ liệu tiêm :", len(missing_vaccination))
display(missing_vaccination.head())

print("\nBệnh nhân thiếu dữ liệu phản ứng :", len(missing_reaction))
display(missing_reaction.head())




Kết quả merge thêm reaction :
reaction_merge
both          260
left_only      17
right_only      0
Name: count, dtype: int64


,patient_id,age_group,gender,vaccine_type,dose_number,vaccination_date,_merge,reaction_type,severity,report_date,reaction_merge
0,BN500,6-17,Female,covid19,2,2024-04-15,both,đau tay,3,27/04/2024,both
1,BN500,6-17,Female,MMR,1st,2024-04-04,both,đau tay,3,27/04/2024,both
2,BN500,6-17,Female,covid19,2,28/02/2024,both,đau tay,3,27/04/2024,both
3,BN500,6-17,Female,DPT,1,19/01/2024,both,đau tay,3,27/04/2024,both
4,BN500,6-17,Female,d.p.t,three,2024-05-11,both,đau tay,3,27/04/2024,both



Bệnh nhân thiếu dữ liệu tiêm : 1


,patient_id
59,BN511



Bệnh nhân thiếu dữ liệu phản ứng : 4


,patient_id
12,BN503
145,BN522
227,BN531
241,BN535


In [6]:

# FULL OUTER MERGE (Root-cause analysis)

full_outer_merge = patients.merge(
    vaccines,
    on="patient_id",
    how="outer",
    indicator=True
).merge(
    reactions,
    on="patient_id",
    how="outer",
    indicator="reaction_merge"
)

print("\nCross-tab of merge indicators (_merge x reaction_merge):")
print(full_outer_merge[["_merge", "reaction_merge"]].value_counts())
display(full_outer_merge.head())

# Phân tích nguyên nhân lệch dữ liệu
missing_vaccination_full = full_outer_merge[full_outer_merge["_merge"] == "left_only"][["patient_id"]].drop_duplicates()
missing_reaction_full = full_outer_merge[full_outer_merge["reaction_merge"] == "left_only"][["patient_id"]].drop_duplicates()

print("\nMissing vaccination (from full outer):", len(missing_vaccination_full))
display(missing_vaccination_full.head())

print("\nMissing reaction (from full outer):", len(missing_reaction_full))
display(missing_reaction_full.head())

# FINAL CLEAN

final_df_clean = final_df.drop(columns=["_merge", "reaction_merge"])
print("\nFinal cleaned dataframe preview:")
display(final_df_clean.head())



Cross-tab of merge indicators (_merge x reaction_merge):
_merge     reaction_merge
both       both              258
           left_only          17
left_only  both                2
Name: count, dtype: int64


,patient_id,age_group,gender,vaccine_type,dose_number,vaccination_date,_merge,reaction_type,severity,report_date,reaction_merge
0,BN500,6-17,Female,covid19,2,2024-04-15,both,đau tay,3,27/04/2024,both
1,BN500,6-17,Female,MMR,1st,2024-04-04,both,đau tay,3,27/04/2024,both
2,BN500,6-17,Female,covid19,2,28/02/2024,both,đau tay,3,27/04/2024,both
3,BN500,6-17,Female,DPT,1,19/01/2024,both,đau tay,3,27/04/2024,both
4,BN500,6-17,Female,d.p.t,three,2024-05-11,both,đau tay,3,27/04/2024,both



Missing vaccination (from full outer): 1


,patient_id
59,BN511



Missing reaction (from full outer): 4


,patient_id
12,BN503
145,BN522
227,BN531
241,BN535



Final cleaned dataframe preview:


,patient_id,age_group,gender,vaccine_type,dose_number,vaccination_date,reaction_type,severity,report_date
0,BN500,6-17,Female,covid19,2,2024-04-15,đau tay,3,27/04/2024
1,BN500,6-17,Female,MMR,1st,2024-04-04,đau tay,3,27/04/2024
2,BN500,6-17,Female,covid19,2,28/02/2024,đau tay,3,27/04/2024
3,BN500,6-17,Female,DPT,1,19/01/2024,đau tay,3,27/04/2024
4,BN500,6-17,Female,d.p.t,three,2024-05-11,đau tay,3,27/04/2024
